In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# torchvision 的输出是[0,1]的PILImage图像，我们把它转换为归一化范围为[-1,1]的张量
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset=torchvision.datasets.CIFAR10(root='./data',train=True,
                                     download=True,transform=transform)
trainloader=torch.utils.data.DataLoader(trainset,batch_size=4,
                                       shuffle=True,num_workers=2)
testset=torchvision.datasets.CIFAR10(root='./data',train=False,
                                    download=True,transform=transform)
testloader=torch.utils.data.DataLoader(testset,batch_size=4,
                                            shuffle=False,num_workers=2)
classes=('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,16*5*5)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

Files already downloaded and verified
Files already downloaded and verified


In [2]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
net=Net().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [8]:
# 训练
for epoch in range(10):
    running_loss=0.0
    for i,data in enumerate(trainloader,0):
        inputs,labels=data
        inputs,labels=inputs.to(device),labels.to(device)
        optimizer.zero_grad() # 清除前一次的梯度
        outputs=net(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss
        if ((i!=0)and(i%10000==0)):
            print("[%d] loss:%.3f"%(epoch+1,running_loss/10000))
            running_loss=0.0
print("Finished Training")

[1] loss:0.770
[2] loss:0.759
[3] loss:0.736
[4] loss:0.722


KeyboardInterrupt: 

In [9]:
# 整体在测试集上的结果
correct=0
total=0
with torch.no_grad():
    for data in testloader:
        images,labels=data
        images,labels=images.to(device),labels.to(device)
        outputs=net(images)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct += (predicted==labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % 
      (100*correct/total))

Accuracy of the network on the 10000 test images: 60 %
